# Classification with perceptron

### Dataset

Using MNIST (the one inside sklearn, with dimensions 8*8)

In [1]:
from sklearn.datasets import load_digits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
mnist = load_digits()
data=pd.DataFrame(mnist.data)
target=pd.DataFrame(mnist.target)

In [114]:
class_1 = 1
class_2 = 7

# Create a binary classification dataset by selecting only the rows that correspond to class_1 and class_2
data_bin = data[(mnist.target == class_1) | (mnist.target == class_2)]
target_bin = target[(mnist.target == class_1) | (mnist.target == class_2)]
target_bin = (target_bin == class_1).astype(int)  # Convert the labels to 0 or 1
data_bin.shape,target_bin.shape

((361, 64), (361, 1))

No need to reshape images as the shape is already flat. Now split into train-test:

In [115]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_bin, target_bin, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((288, 64), (73, 64), (288, 1), (73, 1))

In [116]:
X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

### Perceptron:

In [117]:
# https://towardsdatascience.com/perceptron-learning-algorithm-d5db0deab975

class perceptron:
    def __init__(self):
        self.weights=0
        # x0 will be added to the input matrix and bias will be position 0 (x_0 = 1, W*X = W_0*X_0 + W_1*x_1 +...)

    def init_weights(self,shape):
        W=np.random.rand(shape)
        W=W.reshape(-1,1)
        # print(W.shape)
        return W
    
    def fit(self,data,labels,iters,label_=None):
        self.weights=self.init_weights(data.shape[1]+1)
        # print("W",self.weights.shape)
        x0=np.ones((data.shape[0],1))    #add x0 so the dot product can be computed
        X=np.column_stack((x0,data))
        # print(X[0,:])
        for i in range(iters):
            ix=np.random.choice(X.shape[0],size=1)
            x=X[ix,:].T
            # print("x",x.shape)
            dot=np.dot(self.weights.T,x)
            # print("dot",dot.shape)
            if label_ is not None:
                print(label_[ix])
            if labels[ix] and dot<0:
                self.weights+=x
            if not labels[ix] and dot>=0:
                self.weights-=x
        return


    def predict(self,data):
        predictions=np.where(np.dot(data, self.weights[1:]) + self.weights[0] <= 0, -1, 1)
        return predictions

In [130]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)
# print(y_train[0,:])

p=perceptron()
p.fit(X_train,y_train,iters=100)

In [131]:
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test,p.predict(X_test))
print("Accuracy: ",accuracy)

Accuracy:  0.547945205479452
